# Setup

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf
import random

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure
from skimage import io

# Image Processing

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
def preprocess_image(img):
    equ_img = exposure.equalize_adapthist(img/255, clip_limit=0.05, kernel_size=24)
    return equ_img

In [ ]:
folder_path = "../input/siim-covid19-detection"
train_path = folder_path + "/train/"
test_path = folder_path + "/test/"
    
train_image_data = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
#display(train_image_data)
#train_image_data = train_image_data.drop(labels=['boxes', 'label'], axis=1)
train_image_data = train_image_data.rename(columns = {'id':'image_id', "StudyInstanceUID": "study_id"})
train_image_data["study_id"] = [item + "_study" for item in train_image_data['study_id']]
train_image_data = train_image_data[["study_id", "image_id", "boxes", "label"]]

image_paths_arr = []
for i in range(0, len(train_image_data)):
    study_path = train_path + str(train_image_data['study_id'][i][:-6])
    series = os.listdir(study_path)
    for j in range(0, len(series)):
        series_path = study_path + "/" + series[j]
        images = os.listdir(series_path)
        for k in range(0, len(images)):
            if images[k][:-4] in train_image_data['image_id'][i]:
                image_path = series_path + "/" + images[k]
                image_paths_arr.append(image_path)
                
train_image_data['path'] = image_paths_arr

train_study_data = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

variables = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
for var in variables:
    train_image_data[var] = ""
    for i in range(0, len(train_image_data)):
        train_image_data[var][i] = int(train_study_data[var][train_study_data.index[train_study_data['id'] == train_image_data['study_id'][i]]])
        
train_image_data['Final Label'] = ""
for i in range(0, len(train_image_data)):
    for j in range(0, 4):
        if (train_image_data[variables[j]][i] == 1):
            train_image_data['Final Label'][i] = j

train_image_data = train_image_data[['path', 'study_id', 'image_id', 'Final Label', 'boxes', 'label']]
        
display(train_image_data)

In [ ]:
this_image_info = train_image_data[10:20]
display(this_image_info)

In [ ]:
path = this_image_info['path'][10]

In [ ]:
os.mkdir('/kaggle/working/normaldir')

In [ ]:
xray = read_xray(path)
im = resize(xray, size=600)  
im = np.array(im)
io.imsave('/kaggle/working/normaldir/normal.png', im)
#im = preprocess_image(im)
#io.imsave('test2.png', im)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

def get_bbox (image_path, save_path):

    thickness = 10

    #print(image_path)
    img = dicom2array(path=image_path)
    img = np.stack([img, img, img], axis=-1)
    
    l = this_image_info.loc[this_image_info['path'] == image_path].values[0][5]
    if 'none' in l:
        #plt.figure()
        #plt.imshow(img)
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.save(save_path)
        return
    
    b = this_image_info.loc[this_image_info['path'] == image_path].values[0][4][1:-1]
    b = b.split('}, {')
    for i in range(len(b)):
        if b[i][0] is not '{':
            b[i] = '{' + b[i]
        if b[i][-1] is not '}':
            b[i] = b[i] + '}'
        b[i] = b[i].replace("\'", "\"")
        b[i] = json.loads(b[i])
    
    for i in b:
        img = cv2.rectangle(img, (int(float(i['x'])), int(float(i['y']))), (int(float(i['x'] + i['width'])), int(float(i['y']+i['height']))), [255,0,0], thickness)
    
    #plt.figure()
    #plt.imshow(img)
    img = tf.keras.preprocessing.image.array_to_img(img)
    img.save(save_path)
    return

In [ ]:
get_bbox(path, 'bbox.jpg')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
import warnings
warnings.filterwarnings('ignore')
import shutil 
import tensorflow as tf
%matplotlib inline


import matplotlib.pylab as pylab
import seaborn as sns
import pprint
import pydicom as dicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import wandb

import PIL
from PIL import Image
from colorama import Fore, Back, Style
viz_counter=0

def create_dir(dir, v=1):
    """
    Creates a directory without throwing an error if directory already exists.
    dir : The directory to be created.
    v : Verbosity
    """
    if not os.path.exists(dir):
        os.makedirs(dir)
        if v:
            print("Created Directory : ", dir)
        return 1
    else:
        if v:
            print("Directory already existed : ", dir)
        return 0

voi_lut=True
fix_monochrome=True

def dicom_dataset_to_dict(filename):
    """Credit: https://github.com/pydicom/pydicom/issues/319
               https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    """
    
    dicom_header = dicom.dcmread(filename) 
    
    #====== DICOM FILE DATA ======
    dicom_dict = {}
    repr(dicom_header)
    for dicom_value in dicom_header.values():
        if dicom_value.tag == (0x7fe0, 0x0010):
            #discard pixel data
            continue
        if type(dicom_value.value) == dicom.dataset.Dataset:
            dicom_dict[dicom_value.name] = dicom_dataset_to_dict(dicom_value.value)
        else:
            v = _convert_value(dicom_value.value)
            dicom_dict[dicom_value.name] = v
      
    del dicom_dict['Pixel Representation']
    
    #====== DICOM IMAGE DATA ======
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom_header.pixel_array, dicom_header)
    else:
        data = dicom_header.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom_header.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    modified_image_data = (data * 255).astype(np.uint8)
    
    return dicom_dict, modified_image_data

def _sanitise_unicode(s):
    return s.replace(u"\u0000", "").strip()

def _convert_value(v):
    t = type(v)
    if t in (list, int, float):
        cv = v
    elif t == str:
        cv = _sanitise_unicode(v)
    elif t == bytes:
        s = v.decode('ascii', 'replace')
        cv = _sanitise_unicode(s)
    elif t == dicom.valuerep.DSfloat:
        cv = float(v)
    elif t == dicom.valuerep.IS:
        cv = int(v)
    else:
        cv = repr(v)
    return cv


import os, fnmatch
def find(pattern, path):
    """Utility to find files wrt a regex search"""
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [ ]:
viz_counter=0

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler


def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def unet(input_size=(256,256,1)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

In [ ]:
model = unet(input_size=(512,512,1))
model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss,
                  metrics=[dice_coef, 'binary_accuracy'])
#model.summary()

In [ ]:
model_weights_path = "../input/unet-lung-segmentation-weights-for-chest-x-rays/cxr_reg_weights.best.hdf5"

model.load_weights(model_weights_path)

Shape_X,Shape_Y=512,512

In [ ]:
os.mkdir('/kaggle/working/segmentdir')

In [ ]:
filenames = './normaldir/normal.png'
modified_image_data= cv2.imread(filenames, 0)
resized_image_data = cv2.resize(modified_image_data,(Shape_Y,Shape_X)) # cv2 has this opposite
# props(resized_image_data)
prep_unet_input_img_1 = resized_image_data.reshape(1,Shape_X,Shape_Y,1)
prep_unet_input_img = (prep_unet_input_img_1-127.0)/127.0
pred_img = model.predict(prep_unet_input_img)
pred_img_preprocessed_1 = np.squeeze(pred_img)
pred_img_preprocessed = (pred_img_preprocessed_1*255>127).astype(np.int8)
# props(pred_img_preprocessed)
# print("Unique Values :",np.unique(pred_img_preprocessed))
res = cv2.bitwise_and(resized_image_data,resized_image_data,mask = pred_img_preprocessed)
file_name = filenames.split("/")[-1]
save_path = '/kaggle/working/segmentdir/segment.png'
cv2.imwrite(save_path,res)

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

In [ ]:
dim = 512 #1024, 256, 'original'
#save_dir = f'/kaggle/tmp/segmented_data/{split}/segmented'
test_dir = '/kaggle/working/segmentdir'
#weights_dir = '/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'
weights_dir = '/kaggle/input/yolov5-segmented/segmentedbest.pt'

#shutil.copytree('/kaggle/input/yolov5-repo/yolov5-master', '/kaggle/working/yolov5')
#os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python ../input/yolov5-repo/yolov5-master/detect.py --weights $weights_dir\
--img 512\
--conf 0.1\
--iou 0.5\
--augment\
--source $test_dir\
--save-txt --save-conf --exist-ok
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes
image_ids = []
PredictionStrings = []



In [ ]:
dim = 512 #1024, 256, 'original'
#save_dir = f'/kaggle/tmp/segmented_data/{split}/segmented'
test_dir = '/kaggle/working/normaldir'
#weights_dir = '/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'
weights_dir = '/kaggle/input/weights-of-yolov5-150-epochs/best.pt'

#shutil.copytree('/kaggle/input/yolov5-repo/yolov5-master', '/kaggle/working/yolov5')
#os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python ../input/yolov5-repo/yolov5-master/detect.py --weights $weights_dir\
--img 512\
--conf 0.1\
--iou 0.5\
--augment\
--source $test_dir\
--save-txt --save-conf --exist-ok
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes
image_ids = []
PredictionStrings = []